
# CS436 Project: 3D Scene Reconstruction
## Week 3: Pairwise Colored PLYs

This version reverts to Week 2's two-view pipeline to generate **colored pairwise point clouds**. For each image pair we save a colored PLY in `ply_pairs_colored/` following the naming convention `pair_XX_YY.ply`.



**Pipeline (Pairwise, Colored)**
- Load images from `../data` (optionally downscale) and compute intrinsics from image size.
- Build image pairs (consecutive by default; optional loop closure or all-pairs).
- For each pair (Week 2 approach): dense SIFT+ORB matching → Essential matrix (tight RANSAC) → recover pose → triangulate → PnP refine.
- Save colored PLYs to `ply_pairs_colored/` as `pair_XX_YY.ply`.
- Optional: merge all colored pairs and visualize / ICP refine.


In [1]:

import os, sys, gc
import numpy as np
import cv2

PROJECT_ROOT = os.path.abspath(os.path.join('..'))
SRC_DIR = os.path.join(PROJECT_ROOT, 'src')
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

from utils import (
    load_images_from_folder,
    get_dense_combined_matches,
    sample_colors,
    save_colored_point_cloud_to_ply,
    save_point_cloud_to_ply,
    set_global_memory_mode,
    filter_points,
)
from reconstruction import (
    get_intrinsic_matrix,
    calculate_essential_matrix_from_points,
    recover_camera_pose,
    triangulate_points,
    reconstruct_pair,
    reconstruct_range,
)

set_global_memory_mode()
print('Imports ready; memory mode on.')


Imports ready; memory mode on.


## Configuration

In [2]:
DATA_DIR = '../data'
PAIR_OUTPUT_DIR = 'ply_pairs_colored'   # colored pairwise PLYs inside notebooks/
MAX_VIEWS = None                        # set int to limit number of images (e.g., 20)
DOWNSCALE_MAX_SIDE = 1280               # None to disable; tradeoff: more detail vs RAM

# Pairing options
USE_ALL_PAIRS = False    # True = every combination; False = consecutive only
CLOSE_LOOP = True        # if True and consecutive mode, also add (last, first)

# Reconstruction tunables
RANSAC_THRESH = 1.5             # px threshold for Essential matrix
MIN_COMBINED_MATCHES = 40
MIN_INLIERS = 25
MIN_CHEIRALITY = 15
MAX_DIST_PERCENTILE = 98        # drop extreme far points
VOXEL_SIZE = 0.0                # set >0 (e.g., 0.01) to downsample merged cloud
OUTLIER_STD = 2.0               # statistical outlier removal std ratio

os.makedirs(PAIR_OUTPUT_DIR, exist_ok=True)
print(f'Pair outputs will be saved under {PAIR_OUTPUT_DIR}/')


Pair outputs will be saved under ply_pairs_colored/


## Load images

In [3]:
import glob

def load_images_color(folder_path, target_width=1024):
    imgs = []
    filenames = sorted([f for f in os.listdir(folder_path) if f.lower().endswith(('.png','.jpg','.jpeg','.bmp'))])
    for fname in filenames:
        path = os.path.join(folder_path, fname)
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        if img is None:
            continue
        h, w = img.shape[:2]
        if target_width and w > target_width:
            scale = target_width / w
            new_h = int(h * scale)
            img = cv2.resize(img, (target_width, new_h), interpolation=cv2.INTER_AREA)
        imgs.append(img)
    print(f"Loaded {len(imgs)} color images from {folder_path} (max width {target_width})")
    return imgs

images = load_images_color(DATA_DIR, target_width=DOWNSCALE_MAX_SIDE or 1024)
if MAX_VIEWS is not None:
    images = images[:MAX_VIEWS]
print(f'Total images loaded: {len(images)}')
assert len(images) >= 2, 'Need at least two images to reconstruct.'

K = get_intrinsic_matrix(images[0].shape)
print('Intrinsic matrix (approx):')
print(K)


Loaded 33 color images from ../data (max width 1280)
Total images loaded: 33
Intrinsic matrix (approx):
[[1280.    0.  640.]
 [   0. 1280.  853.]
 [   0.    0.    1.]]


## Build pairs

In [4]:

num_images = len(images)
print(f'Planning pairs for {num_images} images...')

pairs = []
if USE_ALL_PAIRS:
    for i in range(num_images):
        for j in range(i+1, num_images):
            pairs.append((i, j))
else:
    for i in range(num_images - 1):
        pairs.append((i, i+1))
    if CLOSE_LOOP and num_images > 2:
        pairs.append((num_images - 1, 0))

print(f'Total pairs: {len(pairs)}')
print(pairs)


Planning pairs for 33 images...
Total pairs: 33
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 31), (31, 32), (32, 0)]


## Run pairwise colored reconstruction and save PLYs

In [5]:

# Run pairwise colored reconstruction and save PLYs
pair_summaries = []

for i, j in pairs:
    pts3d, cols, pose = reconstruct_pair(
        images[i], images[j], K, i, j,
        min_matches=MIN_COMBINED_MATCHES,
        min_inliers=MIN_INLIERS,
        min_cheirality=MIN_CHEIRALITY,
        ransac_thresh=RANSAC_THRESH,
        max_dist_percentile=MAX_DIST_PERCENTILE
    )

    base = f'pair_{i:02d}_{j:02d}.ply'
    ply_path = os.path.join(PAIR_OUTPUT_DIR, base)

    if pts3d is not None and cols is not None:
        try:
            save_colored_point_cloud_to_ply(pts3d, cols, ply_path)
        except Exception:
            save_point_cloud_to_ply(pts3d, ply_path)
        print(f'✅ Saved {ply_path} ({len(pts3d)} points).')
        pair_summaries.append(((i, j), len(pts3d)))
    else:
        print(f'✗ No output for pair {i}-{j}.')
        pair_summaries.append(((i, j), 0))

print('\n=== Pair Reconstruction Summary ===')
for (i, j), n in pair_summaries:
    status = '✓' if n > 0 else '✗'
    print(f'  {status} ({i},{j}) -> {n}')


  Pair 0 -> 1
    Matches: SIFT=2612 ORB=2075 Combined=4687
    RANSAC inliers: 3247
    Cheirality passed: 3247
    Pose refined via PnP (inliers: 3118)
    Reconstructed points: 3118
    Matches: SIFT=2612 ORB=2075 Combined=4687
    RANSAC inliers: 3247
    Cheirality passed: 3247
    Pose refined via PnP (inliers: 3118)
    Reconstructed points: 3118
Saved colored point cloud to ply_pairs_colored\pair_00_01.ply
✅ Saved ply_pairs_colored\pair_00_01.ply (3118 points).
  Pair 1 -> 2
Saved colored point cloud to ply_pairs_colored\pair_00_01.ply
✅ Saved ply_pairs_colored\pair_00_01.ply (3118 points).
  Pair 1 -> 2
    Matches: SIFT=2320 ORB=1568 Combined=3888
    RANSAC inliers: 2708
    Cheirality passed: 2708
    Pose refined via PnP (inliers: 2599)
    Reconstructed points: 2599
Saved colored point cloud to ply_pairs_colored\pair_01_02.ply
✅ Saved ply_pairs_colored\pair_01_02.ply (2599 points).
  Pair 2 -> 3
    Matches: SIFT=2320 ORB=1568 Combined=3888
    RANSAC inliers: 2708
    Ch

## Merge colored pair clouds into one PLY

In [6]:

MERGED_PLY = os.path.join(PAIR_OUTPUT_DIR, 'reconstruction_pairs_colored_merged.ply')

try:
    import open3d as o3d
    HAS_O3D = True
except ImportError:
    HAS_O3D = False
    print('⚠️  Open3D not installed. Merging will be skipped.')
    print('   Install with: pip install open3d')

if HAS_O3D:
    import glob
    
    # Find all colored pair PLYs
    ply_files = sorted([f for f in glob.glob(os.path.join(PAIR_OUTPUT_DIR, 'pair_*.ply'))])
    
    if not ply_files:
        print('No pair PLYs found to merge.')
    else:
        print(f'Merging {len(ply_files)} pair clouds...')
        pts_list, col_list = [], []
        
        for f in ply_files:
            pcd = o3d.io.read_point_cloud(f)
            pts = np.asarray(pcd.points)
            if pts.size == 0:
                print(f'  Skipped {os.path.basename(f)} (empty)')
                continue
            pts_list.append(pts)
            if pcd.has_colors():
                col_list.append(np.asarray(pcd.colors))
            else:
                col_list.append(np.full((pts.shape[0], 3), 0.5, dtype=np.float64))
            print(f'  Loaded {os.path.basename(f)}: {len(pts)} points')
        
        if not pts_list:
            print('No valid point clouds to merge.')
        else:
            merged_pts = np.vstack(pts_list)
            merged_cols = np.vstack(col_list)
            
            pcd_merged = o3d.geometry.PointCloud()
            pcd_merged.points = o3d.utility.Vector3dVector(merged_pts)
            pcd_merged.colors = o3d.utility.Vector3dVector(merged_cols)
            
            # Optional: voxel downsampling
            if VOXEL_SIZE and VOXEL_SIZE > 0:
                print(f'Applying voxel downsampling (voxel_size={VOXEL_SIZE})...')
                pcd_merged = pcd_merged.voxel_down_sample(VOXEL_SIZE)
                print(f'  After downsampling: {len(pcd_merged.points)} points')
            
            # Optional: statistical outlier removal
            if OUTLIER_STD and OUTLIER_STD > 0:
                print(f'Removing statistical outliers (std_ratio={OUTLIER_STD})...')
                cl, ind = pcd_merged.remove_statistical_outlier(nb_neighbors=40, std_ratio=OUTLIER_STD)
                pcd_merged = cl
                print(f'  After outlier removal: {len(pcd_merged.points)} points')
            
            o3d.io.write_point_cloud(MERGED_PLY, pcd_merged, write_ascii=True)
            print(f'\n✅ Saved merged cloud: {MERGED_PLY}')
            print(f'   Total points: {len(pcd_merged.points)}')


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Merging 33 pair clouds...
  Loaded pair_00_01.ply: 3118 points
  Loaded pair_01_02.ply: 2599 points
  Loaded pair_02_03.ply: 2545 points
  Loaded pair_03_04.ply: 2690 points
  Loaded pair_04_05.ply: 1974 points
  Loaded pair_05_06.ply: 2832 points
  Loaded pair_06_07.ply: 2894 points
  Loaded pair_07_08.ply: 2441 points
  Loaded pair_08_09.ply: 2549 points
  Loaded pair_09_10.ply: 1726 points
  Loaded pair_10_11.ply: 2183 points
  Loaded pair_05_06.ply: 2832 points
  Loaded pair_06_07.ply: 2894 points
  Loaded pair_07_08.ply: 2441 points
  Loaded pair_08_09.ply: 2549 points
  Loaded pair_09_10.ply: 1726 points
  Loaded pair_10_11.ply: 2183 points
  Loaded pair_11_12.ply: 1735 points
  Loaded pair_12_13.ply: 2286 points
  Loaded pair_13_14.ply: 2498 points
  Loaded pair_14_15.ply: 112 points
  Loaded pair_15_16.ply: 1600

In [7]:
# Optional: preview merged cloud
try:
    import open3d as o3d
    if os.path.exists(MERGED_PLY):
        pcd = o3d.io.read_point_cloud(MERGED_PLY)
        if len(np.asarray(pcd.points)) == 0:
            print('Merged cloud is empty.')
        else:
            bbox = pcd.get_axis_aligned_bounding_box()
            bbox.color = (1, 0.6, 0)
            origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
            print(f'Launching Open3D viewer with {len(pcd.points)} points...')
            print('🎮 Controls:')
            print('   • Left-click + drag = Rotate')
            print('   • Right-click + drag = Pan')
            print('   • Scroll = Zoom')
            print('   • Press H = Show all shortcuts')
            o3d.visualization.draw_geometries([pcd, bbox, origin], 
                                             window_name="Pairwise Colored Reconstruction - Merged Cloud",
                                             width=1400, height=1000)
    else:
        print(f'Merged PLY not found at {MERGED_PLY}. Run merge cell first.')
except ImportError:
    print('Open3D not installed. Skipping visualization.')


Launching Open3D viewer with 57201 points...
🎮 Controls:
   • Left-click + drag = Rotate
   • Right-click + drag = Pan
   • Scroll = Zoom
   • Press H = Show all shortcuts


## Optional: ICP-Based Refinement of Pair Clouds

In [8]:

print("\n=== ICP-Based Alignment for Pair Clouds ===")
try:
    import open3d as o3d
    import glob
    
    # Find all pair PLYs in order
    ply_files = sorted([f for f in glob.glob(os.path.join(PAIR_OUTPUT_DIR, 'pair_*.ply'))])
    
    if not ply_files:
        print("No pair PLYs found. Run the reconstruction cell first.")
    else:
        print(f"Found {len(ply_files)} pair clouds to align.")
        
        # Load all clouds
        clouds = []
        for f in ply_files:
            pcd = o3d.io.read_point_cloud(f)
            if len(np.asarray(pcd.points)) > 0:
                clouds.append(pcd)
                print(f"  Loaded {os.path.basename(f)}: {len(pcd.points)} points")
            else:
                print(f"  Skipped {os.path.basename(f)} (empty)")
        
        if len(clouds) < 2:
            print("Not enough non-empty clouds to align (need at least 2).")
        else:
            # Chain ICP: align each cloud to the previous one
            print("\nStarting chained ICP alignment...")
            transformations = [np.eye(4)]  # First cloud is reference
            
            for i in range(1, len(clouds)):
                src = clouds[i]
                ref = clouds[i-1]
                
                # ICP registration
                reg_result = o3d.pipelines.registration.registration_icp(
                    src, ref,
                    max_correspondence_distance=1.0,  # Adjust based on scale
                    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                    criteria=o3d.pipelines.registration.ICPConvergenceCriteria(
                        relative_fitness=1e-6,
                        relative_rmse=1e-6,
                        max_iteration=50
                    )
                )
                
                # Apply transformation to current cloud
                src.transform(reg_result.transformation)
                transformations.append(reg_result.transformation)
                
                fitness = reg_result.fitness
                rmse = reg_result.inlier_rmse
                print(f"  Pair {i-1}→{i}: fitness={fitness:.4f}, RMSE={rmse:.6f}")
            
            # Merge all aligned clouds
            print("\nMerging ICP-aligned pair clouds...")
            merged_icp = o3d.geometry.PointCloud()
            for cloud in clouds:
                merged_icp += cloud
            
            # Optional: downsample
            if VOXEL_SIZE and VOXEL_SIZE > 0:
                merged_icp = merged_icp.voxel_down_sample(VOXEL_SIZE)
                print(f"Downsampled to {len(merged_icp.points)} points (voxel_size={VOXEL_SIZE}).")
            
            # Optional: outlier removal
            if OUTLIER_STD and OUTLIER_STD > 0:
                cl, ind = merged_icp.remove_statistical_outlier(nb_neighbors=40, std_ratio=OUTLIER_STD)
                merged_icp = cl
                print(f"Removed outliers, {len(merged_icp.points)} points remain.")
            
            # Save refined cloud
            ICP_MERGED_PLY = os.path.join(PAIR_OUTPUT_DIR, 'reconstruction_pairs_icp_refined.ply')
            o3d.io.write_point_cloud(ICP_MERGED_PLY, merged_icp, write_ascii=True)
            print(f"\n✅ Saved ICP-refined merged cloud: {ICP_MERGED_PLY}")
            print(f"   Total points: {len(merged_icp.points)}")
            
            # Visualize result
            print("\nLaunching Open3D viewer for ICP-refined pair cloud...")
            bbox = merged_icp.get_axis_aligned_bounding_box()
            bbox.color = (1, 0.6, 0)
            origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
            o3d.visualization.draw_geometries([merged_icp, bbox, origin], 
                                             window_name="ICP-Refined Pair Clouds",
                                             width=1400, height=1000)
            
except ImportError:
    print("⚠️  Open3D not installed. Skipping ICP refinement.")
    print("   Install with: pip install open3d")
except Exception as e:
    print(f"⚠️  ICP alignment error: {e}")



=== ICP-Based Alignment for Pair Clouds ===
Found 33 pair clouds to align.
  Loaded pair_00_01.ply: 3118 points
  Loaded pair_01_02.ply: 2599 points
  Loaded pair_02_03.ply: 2545 points
  Loaded pair_03_04.ply: 2690 points
  Loaded pair_04_05.ply: 1974 points
  Loaded pair_05_06.ply: 2832 points
  Loaded pair_06_07.ply: 2894 points
  Loaded pair_07_08.ply: 2441 points
  Loaded pair_08_09.ply: 2549 points
  Loaded pair_09_10.ply: 1726 points
  Loaded pair_10_11.ply: 2183 points
  Loaded pair_11_12.ply: 1735 points
  Loaded pair_12_13.ply: 2286 points
  Loaded pair_13_14.ply: 2498 points
  Loaded pair_14_15.ply: 112 points
  Loaded pair_15_16.ply: 1600 points
  Loaded pair_16_17.ply: 1126 points
  Loaded pair_17_18.ply: 1912 points
  Loaded pair_18_19.ply: 1780 points
  Loaded pair_19_20.ply: 1459 points
  Loaded pair_20_21.ply: 1885 points
  Loaded pair_21_22.ply: 231 points
  Loaded pair_22_23.ply: 90 points
  Loaded pair_23_24.ply: 2265 points
  Loaded pair_24_25.ply: 1940 points
  L